In [40]:
import pandas as pd
import numpy as np
import os

import json
import re

In [41]:
data = pd.read_excel('routine.xlsx', sheet_name=2 ,skiprows=4)
data.columns = data.columns.str.replace('\n', '_')
data.columns = data.columns.str.replace(':', '')
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace(r'\.\d+', '', regex=True)
df = data.fillna('-') 
df.columns = df.columns.str.upper()

skips the unnecessary sheets and also skipps the first 4 rows which r the headings of the sheet.

In [42]:
data_cleaning = df.drop(["CLASS_SIZE", "ASSIGNED_HALL", "HALL_CAPACITY_MATCH", "UNNAMED_57"], axis='columns')
# len(data.columns)
indexList = list(range(61))
for i in range(2, 61, 5):
    indexList.remove(i)
clean_data = data_cleaning.iloc[:, indexList]

adds the day in all the rows of the day column 

In [43]:
def fill_days(column):
    current_day = None
    new_column = column.copy()  # Create a copy to avoid warnings
    for i, value in enumerate(new_column):
        if pd.notna(value) and value != '-':
            current_day = value
        elif value == '-' and current_day is not None:
            new_column.iloc[i] = current_day
    return new_column

# Make sure clean_data is a copy, not a view
clean_data = clean_data.copy()

# Apply the function to the 'DAY' column
clean_data.loc[:, 'DAY'] = fill_days(clean_data['DAY'])

this fills in all the columns in day secction. all the columns r filled wit day name

In [44]:
days_of_week = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY']
monday_df, tuesday_df, wednesday_df, thursday_df, friday_df, saturday_df = [clean_data.loc[clean_data['DAY'] == day] for day in days_of_week]

# Create a new folder to store the CSV files
new_folder = "excel data"
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

for day in days_of_week:
    day_data = clean_data[clean_data['DAY'] == day]
    filename = f"{day}.xlsx"  # Create a unique filename for each day
    file_path = os.path.join(new_folder, filename)  # Join the folder path with the filename
    day_data.to_excel(file_path, index=False)


makes indivigual dataframes for every 'day' section of the week

In [45]:

# ! Define the time intervals to group data
time_intervals = ['7.00 - 7.45', '7.45 - 8.30', '8.30 - 9.20', '9.20 - 10.10', '10.10 - 11.00', '11.00 - 11.50', '11.50 - 12.35', '1.00 - 1.45', '1.45 - 2.30', '2.30 - 3.15', '3.15 - 4.00', '4.00 - 4.45']

# Extract columns except 'DAY'
columns_except_day = clean_data.columns[1:]  # Skip 'DAY' column

# Group every 4 columns together and create multi-level column names
n_sets = len(columns_except_day) // 4
new_columns = [('DAY', '')]  # Keep 'DAY' at the start

for i in range(n_sets):
    time_range = time_intervals[i]  # Assign each group a time interval
    new_columns.extend([(time_range, col) for col in ['COURSE_NAME', 'COURSE_CODE_SECTION_&_SHIFT', 'FACULTY_INITIALS', 'HALL']])

# Assign the new MultiIndex to the DataFrame columns
monday_df.columns = pd.MultiIndex.from_tuples(new_columns)

In [46]:

def sort_time_intervals(time_intervals):
    def extract_time(time_interval):
        # Extract the start time as a float for sorting
        start_time = re.match(r'(\d+\.\d+)', time_interval)
        return float(start_time.group(1)) if start_time else float('inf')
    
    # Sort the intervals based on their start time
    return sorted(time_intervals, key=extract_time)

def convert_df_to_json(df):
    # Initialize the dictionary to hold the structured data
    structured_json = {}
    
    # Extract time intervals from the MultiIndex
    time_intervals = df.columns.levels[0]
    
    # Sort the time intervals numerically
    time_intervals = sort_time_intervals(time_intervals)
    
    # Iterate over each time interval
    for time_interval in time_intervals:
        if time_interval in df.columns.levels[0]:
            # Filter DataFrame to get the relevant columns for the current time interval
            interval_df = df[time_interval]
            
            # Ensure interval_df is a DataFrame
            if isinstance(interval_df, pd.Series):
                interval_df = interval_df.to_frame()
            
            # Convert the DataFrame to a list of dictionaries
            records = interval_df.reset_index(drop=True).to_dict(orient='records')
            
            # Add the data to the structured JSON
            structured_json[time_interval] = records
        else:
            # Ensure that even empty time intervals are included in the JSON
            structured_json[time_interval] = []

    # Return the final structured JSON with the day as the key
    return {"monday": structured_json}

# ! Convert monday_df to JSON format
try:
    monday_json = convert_df_to_json(monday_df)

    # ! save it to a .json file
    with open('monday_data.json', 'w') as f:
        json.dump(monday_json, f, indent=4)
    
    print("JSON data saved to monday_data.json!")
except Exception as e:
    print(f"An error occurred: {e}")


JSON data saved to monday_data.json!
